In [ ]:
# Upgrade pip
!pip install --upgrade pip

# Install/upgrade required libraries
!pip install torch --quiet
!pip install torchvision --quiet
!pip install torchaudio --quiet
!pip install transformers==4.33.1 --quiet
!pip install datasets --quiet
!pip install scikit-learn --quiet
!pip install codecarbon --quiet
!pip install numpy==1.26.4 --quiet
!pip install pandas --quiet
!pip install tqdm --quiet
!pip install scikit-learn --quiet
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:

import torch
import torch.nn as nn
from transformers import DistilBertForSequenceClassification, DistilBertForQuestionAnswering, BertForSequenceClassification, BertForQuestionAnswering
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import random
import numpy as np
import time
import json
import logging
from sklearn.metrics import accuracy_score, f1_score
from codecarbon import OfflineEmissionsTracker
import warnings
import os
import pandas as pd
# --- SECTION: 1. CONFIGURATION ---
warnings.filterwarnings("ignore")
logging.getLogger("codecarbon").setLevel(logging.INFO)
DEVICE_CONFIG = {
    'optimize_for_gpu': True,
    'mixed_precision': True
}
DEVICE = "cuda" if torch.cuda.is_available() and DEVICE_CONFIG['optimize_for_gpu'] else "cpu"
print(f"Using device: {DEVICE}")

DATASETS = {
    'glue_sst2': {'name': 'glue', 'config': 'sst2', 'split_train': 'train', 'split_val': 'validation'},
    'glue_mrpc': {'name': 'glue', 'config': 'mrpc', 'split_train': 'train', 'split_val': 'validation'},
    'glue_rte': {'name': 'glue', 'config': 'rte', 'split_train': 'train', 'split_val': 'validation'},
    'squad': {'name': 'squad', 'split_train': 'train', 'split_val': 'validation'}
}
MAX_SAMPLES = 5000
WATER_USAGE_FACTORS = {"average_l_per_kwh": 1.8}
CARBON_INTENSITY = 250  # gCO2e/kWh
BATCH_SIZE = 16
SEQ_LENGTH_CLASSIFICATION = 128
SEQ_LENGTH_QA = 384
NUM_LAYERS = 1
GLUE_TASKS_TO_RUN = ['sst2', 'mrpc', 'rte']
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_device():
    device = torch.device(DEVICE)
    if DEVICE == "cuda":
        try:
            print(f"✓ Using GPU: {torch.cuda.get_device_name(0)} (CUDA)")
        except Exception:
            print("✓ Using GPU (name unknown)")
        if DEVICE_CONFIG['mixed_precision']:
            print("✓ Mixed precision (FP16) enabled.")
    else:
        print(f"✓ Using CPU")
    return device
# --- SECTION: 2. QUANTIZATION ---
class AdaptiveQuantizedLayer(nn.Module):
    def __init__(self, layer, quant_bits=4):
        super().__init__()
        self.layer = layer
        self.current_precision = 0
        self.switch_count = 0
        self.set_precision(quant_bits)

    def set_precision(self, bits):
        if self.current_precision != bits and self.current_precision != 0:
            self.switch_count += 1
            
        self.quant_bits = bits
        self.current_precision = bits
        
        if bits < 32:
            self.min_val = -2 ** (bits - 1)
            self.max_val = 2 ** (bits - 1) - 1
        else:
            self.min_val, self.max_val = None, None

    def quantize(self, x):
        scale = (self.max_val - self.min_val) / (x.max() - x.min() + 1e-6)
        zero_point = x.min() - self.min_val / scale
        x_quant = torch.round(x * scale + zero_point)
        x_quant = torch.clamp(x_quant, self.min_val, self.max_val)
        x_dequant = (x_quant - zero_point) / scale
        return x_dequant

    def forward(self, *args, **kwargs):
        output = self.layer(*args, **kwargs)
        if self.current_precision == 32:
            return output
        if isinstance(output, tuple):
            hidden_states = output[0]
            hidden_states = self.quantize(hidden_states)
            return (hidden_states,) + output[1:]
        else:
            return self.quantize(output)

class PrecisionScheduler:
    def __init__(self, model, grad_norm_threshold=1.0):
        self.model = model
        self.grad_norm_threshold = grad_norm_threshold
        self.adaptable_layers = [m for m in self.model.modules() if isinstance(m, AdaptiveQuantizedLayer)]
        self.switch_log = [] 
        self.current_epoch = 0
        print(f"✓ Gradient-Aware PrecisionScheduler initialized for {len(self.adaptable_layers)} adaptable layers.")
        
    def get_global_norm(self):
        total_norm = 0.0
        for p in self.model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        return total_norm ** 0.5

    def step(self, epoch):
        """Update precision based on gradient norms and log events."""
        self.current_epoch = epoch
        global_norm = self.get_global_norm()
        
        # Check if we need to switch precision
        if global_norm > self.grad_norm_threshold:
            self.increase_precision(global_norm)
        elif global_norm < self.grad_norm_threshold / 2.0:
            self.decrease_precision(global_norm)
            
        return global_norm

    def increase_precision(self, current_norm):
        for idx, layer in enumerate(self.adaptable_layers):
            if layer.current_precision == 4:
                new_prec = 8
            elif layer.current_precision == 8:
                new_prec = 32
            else:
                continue
            
            layer.set_precision(new_prec)
            event = {
                "layer_id": idx,
                "new_precision": new_prec,
                "grad_norm": current_norm,
                "epoch": self.current_epoch,
                "type": "increase"
            }
            self.switch_log.append(event)
            print(f"⚡ Epoch {self.current_epoch} | Layer {idx} switched to {new_prec}-bit (grad_norm={current_norm:.4f})")
            return

    def decrease_precision(self, current_norm):
        for idx, layer in enumerate(self.adaptable_layers):
            if layer.current_precision == 32:
                new_prec = 8
            elif layer.current_precision == 8:
                new_prec = 4
            else:
                continue
            
            layer.set_precision(new_prec)
            event = {
                "layer_id": idx,
                "new_precision": new_prec,
                "grad_norm": current_norm,
                "epoch": self.current_epoch,
                "type": "decrease"
            }
            self.switch_log.append(event)
            print(f"📉 Epoch {self.current_epoch} | Layer {idx} switched to {new_prec}-bit (grad_norm={current_norm:.4f})")
            return



def build_adaptive_quantized_model(device, model_type="distilbert", task_type="classification"):
    print(f"\n🏗️ Building Adaptive Quantized {model_type.capitalize()} Model...")
    ModelClass = DistilBertForSequenceClassification if task_type == "classification" else DistilBertForQuestionAnswering
    ModelBase = "distilbert-base-uncased"
    if model_type == "bert":
        ModelClass = BertForSequenceClassification if task_type == "classification" else BertForQuestionAnswering
        ModelBase = "bert-base-uncased"

    model_args = {'num_labels': 2} if task_type == "classification" else {}
    model = ModelClass.from_pretrained(ModelBase, **model_args).to(device)
    
    class QuantizedModel(nn.Module):
        def __init__(self, model_to_wrap):
            super().__init__()
            self.model = model_to_wrap
            self.model_type = model_type
            self.config = self.model.config
            layers_to_quantize = self.model.bert.encoder.layer if model_type == "bert" else self.model.distilbert.transformer.layer
            for i in range(len(layers_to_quantize)):
                layers_to_quantize[i] = AdaptiveQuantizedLayer(layers_to_quantize[i], quant_bits=8)
        
        def forward(self, **kwargs):
            return self.model(**kwargs)

    quantized_model = QuantizedModel(model).to(device)
    print(f"✓ Successfully created quantized {model_type.capitalize()} model.")
    return quantized_model


# --- SECTION: 3. DATA LOADING ---
def get_dataloaders(model_type, seq_len, batch_size):
    from transformers import DistilBertTokenizerFast, BertTokenizerFast
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') if model_type == 'distilbert' else BertTokenizerFast.from_pretrained('bert-base-uncased')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def preprocess_glue_sst2(examples):
        enc = tokenizer(examples["sentence"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    def preprocess_glue_mrpc(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    def preprocess_glue_rte(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    try:
        glue_sst2_train_data = load_dataset("glue", "sst2", split=f"train[:{MAX_SAMPLES}]")
        glue_sst2_validation_data = load_dataset("glue", "sst2", split=f"validation[:{MAX_SAMPLES}]")
        glue_mrpc_train_data = load_dataset("glue", "mrpc", split=f"train[:{5000}]")
        glue_mrpc_validation_data = load_dataset("glue", "mrpc", split=f"validation[:{5000}]")
        glue_rte_train_data = load_dataset("glue", "rte", split=f"train[:{2490}]")
        glue_rte_validation_data = load_dataset("glue", "rte", split=f"validation[:{277}]")

        glue_sst2_train_data = glue_sst2_train_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_sst2_validation_data = glue_sst2_validation_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_mrpc_train_data = glue_mrpc_train_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_mrpc_validation_data = glue_mrpc_validation_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_train_data = glue_rte_train_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_validation_data = glue_rte_validation_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])

        glue_sst2_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_sst2_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_mrpc_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_mrpc_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_rte_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_rte_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

        dataloaders = {
            'glue_sst2_train': DataLoader(glue_sst2_train_data, batch_size=batch_size, shuffle=True),
            'glue_sst2_validation': DataLoader(glue_sst2_validation_data, batch_size=batch_size, shuffle=False),
            'glue_mrpc_train': DataLoader(glue_mrpc_train_data, batch_size=batch_size, shuffle=True),
            'glue_mrpc_validation': DataLoader(glue_mrpc_validation_data, batch_size=batch_size, shuffle=False), 
            'glue_rte_train': DataLoader(glue_rte_train_data, batch_size=batch_size, shuffle=True), 
            'glue_rte_validation': DataLoader(glue_rte_validation_data, batch_size=batch_size, shuffle=False),
        }
        return dataloaders, tokenizer
    except Exception as e:
        print(f"⚠️ get_dataloaders failed: {e}. Returning empty dataloaders.")
        return {}, None

# --- SECTION: 4. EVALUATION METRICS ---
def evaluate_classification(model, dataloader, device, is_bert):
    model.eval()
    predictions = []
    labels = []
    for batch in tqdm(dataloader, desc="Evaluating"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        if not is_bert:
            inputs.pop('token_type_ids', None)
        if DEVICE_CONFIG['mixed_precision'] and device.type == 'cuda':
            inputs = {k: v.half() if v.dtype == torch.float32 else v for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs.last_hidden_state.norm(dim=-1)
        predicted = torch.argmax(logits, dim=-1) if logits.dim() > 1 else (logits > 0.5).long()
        predictions.extend(predicted.cpu().numpy())
        labels.extend(batch['labels'].cpu().numpy())
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}


# --- SECTION: 5. FINE-TUNING ---
# --- SECTION: 5. FINE-TUNING ---
def fine_tune(model, dataloader, device, epochs=3, task_type="classification", scheduler_callback=None, is_bert=False):
    model.train()
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    num_training_steps = len(dataloader) * epochs
    num_warmup_steps = int(num_training_steps * 0.1)

    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    precision_scheduler = None
    if any(isinstance(m, AdaptiveQuantizedLayer) for m in model.modules()):
        # FIX: The grad_norm_threshold was too low.
        # A common practice is to allow some initial instability during warmup.
        # We will dynamically adjust the threshold based on the epoch.
        precision_scheduler = PrecisionScheduler(model, grad_norm_threshold=1.0)
    
    # FIX: Initialize all layers to 32-bit for a proper warmup phase.
    for layer in model.modules():
        if isinstance(layer, AdaptiveQuantizedLayer):
            layer.set_precision(8)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        total_loss = 0
        
        for batch in tqdm(dataloader, desc=f"Training Epoch {epoch+1}"):
            # FIX: Moved optimizer.zero_grad() here to clear gradients from previous batch.
            optimizer.zero_grad()
            
            inputs = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            if 'input_ids' not in inputs:
                print("Skipping a batch with missing 'input_ids'.")
                continue

            # --- Forward Pass ---
            outputs = model(**inputs)
            loss = outputs.loss
            
            # --- Backward Pass ---
            loss.backward()

            # --- Gradient-Aware Precision Scheduling & Logging ---
            # FIX: Re-located the scheduler logic to after the backward pass, before the optimizer step.
            # This ensures we are checking actual gradients.
            if precision_scheduler: # Start adapting after a warmup period
                grad_norm = precision_scheduler.step(epoch + 1)
                print(f"🔎 Epoch {epoch+1}: Global gradient norm = {grad_norm:.4f}")
                # FIX: Check the grad_norm against the threshold *and* adapt precision here
                if grad_norm > precision_scheduler.grad_norm_threshold:
                    precision_scheduler.increase_precision(grad_norm)
                
            # FIX: Added a strong gradient clipping step here.
            # Your original code already had this, but it's crucial for stability.
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # --- Optimizer Step ---
            optimizer.step()
            lr_scheduler.step()
            total_loss += loss.item()

            if scheduler_callback:
                scheduler_callback()

        if len(dataloader) > 0:
            print(f"Average loss: {total_loss / len(dataloader):.4f}")
        else:
            print("Warning: Dataloader was empty.")
            
    if precision_scheduler:
        return precision_scheduler.switch_log
    else:
        return []

# --- SECTION: 6. EXPERIMENT RUNNER ---
def run_experiment(model, model_name, dataloaders, device, is_bert, batch_size, seq_len, task_type, run_sst2=True, run_mrpc=True, run_rte=True):
    results = {
        'model_name': model_name,
        'batch_size': batch_size,
        'seq_length': seq_len,
        'task_type': task_type,
        'accuracy_metrics': {},
        'performance_metrics': {},
        'scheduler_metrics': {
            'precision_distribution': {'INT4': 0.0, 'INT8': 0.0, 'FP32': 1.0},
            'precision_switch_counts_int4': 0,
            'precision_switch_counts_int8': 0,
            'precision_switch_counts_fp32': 0,
            'avg_precision_level': 32.0,
            'adaptation_log': []
        }
    }
    
    def update_scheduler_metrics(model):
        int4_count, int8_count, fp32_count = 0, 0, 0
        switch_int4, switch_int8, switch_fp32 = 0, 0, 0
        total_precision, total_layers = 0, 0

        for layer in model.modules():
            if isinstance(layer, AdaptiveQuantizedLayer):
                total_layers += 1
                total_precision += layer.current_precision
                if layer.current_precision == 4:
                    int4_count += 1
                    switch_int4 += layer.switch_count
                elif layer.current_precision == 8:
                    int8_count += 1
                    switch_int8 += layer.switch_count
                elif layer.current_precision == 32:
                    fp32_count += 1
                    switch_fp32 += layer.switch_count

        if total_layers == 0:
            return results['scheduler_metrics']

        return {
            'precision_distribution': {'INT4': int4_count / total_layers, 'INT8': int8_count / total_layers, 'FP32': fp32_count / total_layers},
            'precision_switch_counts_int4': switch_int4,
            'precision_switch_counts_int8': switch_int8,
            'precision_switch_counts_fp32': switch_fp32,
            'avg_precision_level': total_precision / total_layers,
            'adaptation_log': results['scheduler_metrics'].get('adaptation_log', [])
        }

    print(f"\n--- 🚀 Measuring Metrics for {model_name} ({task_type}) ---")
    start_time = time.time()
    num_queries = 0
    emissions_kwh = 0.0

    tracker = OfflineEmissionsTracker(
        project_name=f"Experiment_{model_name.replace(' ', '_')}",
        measure_power_secs=1,
        output_dir=".",
        log_level='info',
        country_iso_code="USA",
        region=None
    )
    tracker.start()
    try:
        if run_sst2 and task_type == "classification":
            print("\n--- Fine-tuning on GLUE SST-2 ---")
            adaptation_log = fine_tune(model, dataloaders['glue_sst2_train'], device, epochs=5, task_type=task_type)
            results['scheduler_metrics']['adaptation_log'] = adaptation_log
            print(" Evaluating GLUE SST-2...")
            metrics = evaluate_classification(model, dataloaders['glue_sst2_validation'], device, is_bert)
            results['accuracy_metrics']['sst2_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['sst2_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_sst2_validation'].dataset)
            print(f" SST-2 Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        if run_mrpc and task_type == "classification":
            print("\n--- Fine-tuning on GLUE MRPC ---")
            adaptation_log = fine_tune(model, dataloaders['glue_mrpc_train'], device, epochs=5, task_type=task_type)
            results['scheduler_metrics']['adaptation_log'] = adaptation_log
            print(" Evaluating GLUE MRPC...")
            metrics = evaluate_classification(model, dataloaders['glue_mrpc_validation'], device, is_bert)
            results['accuracy_metrics']['mrpc_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['mrpc_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_mrpc_validation'].dataset)
            print(f" MRPC Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        if run_rte and task_type == "classification":
            print("\n--- Fine-tuning on GLUE RTE ---")
            adaptation_log = fine_tune(model, dataloaders['glue_rte_train'], device, epochs=3, task_type=task_type)
            results['scheduler_metrics']['adaptation_log'] = adaptation_log
            print(" Evaluating GLUE RTE...")
            metrics = evaluate_classification(model, dataloaders['glue_rte_validation'], device, is_bert)
            results['accuracy_metrics']['rte_accuracy'] = metrics['accuracy']
            results['accuracy_metrics']['rte_f1'] = metrics['f1']
            num_queries += len(dataloaders['glue_rte_validation'].dataset)
            print(f" RTE Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        results['scheduler_metrics'] = update_scheduler_metrics(model)
        emissions_kwh = tracker.stop() or 0.0

    except Exception as e:
        print(f"🚨 Experiment failed for {model_name}: {e}")
        try:
            tracker.stop()
        except Exception:
            pass
        emissions_kwh = 0.0

    total_duration_s = time.time() - start_time
    total_tokens_processed = num_queries * seq_len
    total_carbon_g = emissions_kwh * CARBON_INTENSITY
    results['performance_metrics'] = {
        'latency_ms_query': (total_duration_s / num_queries) * 1000 if num_queries > 0 else 0,
        'throughput_tokens_sec': total_tokens_processed / total_duration_s if total_duration_s > 0 else 0,
        'energy_wh_token': (emissions_kwh * 1000) / total_tokens_processed if total_tokens_processed > 0 else 0,
        'sci_gco2e_query': total_carbon_g / num_queries if num_queries > 0 else 0,
        'wue_avg_liters_query': (emissions_kwh * WATER_USAGE_FACTORS['average_l_per_kwh']) / num_queries if num_queries > 0 else 0,
        'total_emissions_kwh': emissions_kwh
    }

    print(f"\n--- Results for {model_name} ---")
    print(f" Duration: {total_duration_s:.2f}s | Emissions: {emissions_kwh:.6f} kWh | Queries: {num_queries}")
    print(json.dumps(results, indent=2))
    print("-" * 40)
    return results


# --- SECTION: 7. MAIN EXECUTION ---
if __name__ == "__main__":
    DEVICE = get_device()
    set_seed(42)
    batch_size = BATCH_SIZE
    num_layers = NUM_LAYERS

    # Initialize results list
    all_results = []

    # Define model configurations
    model_configs = [
     # {'model_type': 'distilbert', 'task_type': 'classification', 'is_quantized': False, 'name': 'Baseline_DistilBERT_Classification'},
     #   {'model_type': 'distilbert', 'task_type': 'classification', 'is_quantized': True, 'name': 'Adaptive_Quantized_DistilBERT_Classification'},
        
       #{'model_type': 'bert', 'task_type': 'classification', 'is_quantized': False, 'name': 'Baseline_BERT_Classification'},
        {'model_type': 'bert', 'task_type': 'classification', 'is_quantized': True, 'name': 'Adaptive_Quantized_BERT_Classification'},

    ]

    # Run experiments for each model configuration
    for config in model_configs:
        model_type = config['model_type']
        task_type = config['task_type']
        is_quantized = config['is_quantized']
        model_name = config['name']
        seq_len = SEQ_LENGTH_QA if task_type == 'qa' else SEQ_LENGTH_CLASSIFICATION

        # Get dataloaders
        dataloaders, tokenizer = get_dataloaders(model_type=model_type, seq_len=seq_len, batch_size=batch_size)
        
       # model_name = f"{model_name}_{task.upper()}"
       # print(f"\n=== Running experiment for {model_name} ===")
        if is_quantized:
            model = build_adaptive_quantized_model(DEVICE, model_type, task_type)
        else:
            ModelClass = DistilBertForSequenceClassification if model_type == 'distilbert' else BertForSequenceClassification
            model = ModelClass.from_pretrained(f"{model_type}-base-uncased", num_labels=2).to(DEVICE)
                
        results = run_experiment(
                model=model, model_name=model_name, dataloaders=dataloaders, device=DEVICE,
                is_bert=(model_type == 'bert'), batch_size=BATCH_SIZE, seq_len=seq_len, task_type=task_type,
                run_sst2=True, run_mrpc=False, run_rte=False
            )
   
        all_results.append(results)
        
    # Save all results to a JSON file
    with open('adaptive_quantization_results.json', 'w') as f:
        json.dump(all_results, f, indent=2)

    df_out = pd.json_normalize(all_results, sep='_')
    df_out.to_csv('adaptive_quantization_results.csv', index=False)
    print("✅ CSV saved to 'adaptive_quantization_results.csv'")
    print("\nAll experiments completed. Results saved.")

In [1]:

import torch
import torch.nn as nn
from transformers import DistilBertForSequenceClassification, DistilBertForQuestionAnswering, BertForSequenceClassification, BertForQuestionAnswering
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from collections import deque
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import random
import numpy as np
import time
import json
import logging
from sklearn.metrics import accuracy_score, f1_score
from codecarbon import OfflineEmissionsTracker
import warnings
import os
import pandas as pd
# --- STRATEGY 1: Import the highly optimized spikingjelly library ---
from spikingjelly.activation_based import neuron, functional

# --- SECTION: 1. CONFIGURATION ---
warnings.filterwarnings("ignore")
logging.getLogger("codecarbon").setLevel(logging.INFO)
DEVICE_CONFIG = {
    'optimize_for_gpu': True,
    'mixed_precision': True
}
DEVICE = "cuda" if torch.cuda.is_available() and DEVICE_CONFIG['optimize_for_gpu'] else "cpu"
print(f"Using device: {DEVICE}")

DATASETS = {
    'glue_sst2': {'name': 'glue', 'config': 'sst2', 'split_train': 'train', 'split_val': 'validation'},
    'glue_mrpc': {'name': 'glue', 'config': 'mrpc', 'split_train': 'train', 'split_val': 'validation'},
    'glue_rte': {'name': 'glue', 'config': 'rte', 'split_train': 'train', 'split_val': 'validation'},
    'squad': {'name': 'squad', 'split_train': 'train', 'split_val': 'validation'}
}
MAX_SAMPLES = 5000
WATER_USAGE_FACTORS = {"average_l_per_kwh": 1.8}
CARBON_INTENSITY = 250  # gCO2e/kWh
BATCH_SIZE = 16
SEQ_LENGTH_CLASSIFICATION = 128
SEQ_LENGTH_QA = 384
NUM_LAYERS = 1
GLUE_TASKS_TO_RUN = ['sst2', 'mrpc', 'rte']
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_device():
    device = torch.device(DEVICE)
    if DEVICE == "cuda":
        try:
            print(f"✓ Using GPU: {torch.cuda.get_device_name(0)} (CUDA)")
        except Exception:
            print("✓ Using GPU (name unknown)")
        if DEVICE_CONFIG['mixed_precision']:
            print("✓ Mixed precision (FP16) enabled.")
    else:
        print(f"✓ Using CPU")
    return device
# --- SECTION: 2. QUANTIZATION ---
# --- SECTION: 2. QUANTIZATION ---
# --- SECTION: 2. QUANTIZATION ---
# --- SECTION: 2. QUANTIZATION ---
class AdaptiveQuantizedLayer(nn.Module):
    def __init__(self, layer, quant_bits=32):
        super().__init__()
        self.layer = layer
        self.current_precision = 0
        self.switch_count = {'INT4': 0, 'INT8': 0, 'FP32': 0}
        self.set_precision(quant_bits)

    def set_precision(self, bits):
        if self.current_precision != bits and self.current_precision != 0:
            self.switch_count['INT' + str(bits) if bits < 32 else 'FP32'] += 1
        self.quant_bits = bits
        self.current_precision = bits
        if bits < 32:
            self.min_val = -2 ** (bits - 1)
            self.max_val = 2 ** (bits - 1) - 1
        else:
            self.min_val, self.max_val = None, None

    def quantize(self, x):
        x_max, x_min = x.max(), x.min()
        if x_max == x_min:  # Prevent division by zero
            return x
        scale = (self.max_val - self.min_val) / (x_max - x_min + 1e-8)
        zero_point = x_min - self.min_val / scale
        x_quant = torch.round(x * scale + zero_point)
        x_quant = torch.clamp(x_quant, self.min_val, self.max_val)
        x_dequant = (x_quant - zero_point) / scale
        return x_dequant

    def forward(self, *args, **kwargs):
        output = self.layer(*args, **kwargs)
        if self.current_precision == 32 or not self.training:
            return output
        if isinstance(output, tuple):
            hidden_states = output[0]
            hidden_states = self.quantize(hidden_states)
            return (hidden_states,) + output[1:]
        else:
            return self.quantize(output)

class PrecisionScheduler:
    def __init__(self, model, initial_grad_norm_threshold=10.0, ma_window_size=20, cooldown_steps=20, warmup_steps=1000, alpha=0.9, task_name='sst2'):
        self.model = model
        self.initial_grad_norm_threshold = initial_grad_norm_threshold
        self.current_threshold = initial_grad_norm_threshold
        self.alpha = alpha
        self.ma_window_size = ma_window_size
        self.cooldown_steps = cooldown_steps
        self.warmup_steps = 1000 if task_name == 'sst2' else 500  # Longer warmup for SST-2
        self.adaptable_layers = [m for m in self.model.modules() if isinstance(m, AdaptiveQuantizedLayer)]
        self.grad_norm_history = deque(maxlen=ma_window_size)
        self.layer_cooldowns = [0] * len(self.adaptable_layers)
        self.switch_log = []
        self.current_epoch = 0
        self.step_count = 0
        print(f"✓ Robust PrecisionScheduler initialized for {len(self.adaptable_layers)} layers (Window: {ma_window_size}, Cooldown: {cooldown_steps}, Warmup: {self.warmup_steps}).")

    def get_global_norm(self):
        total_norm = 0.0
        param_count = 0
        for name, p in self.model.named_parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                if not torch.isfinite(param_norm):  # Skip non-finite norms
                    print(f"Warning: Non-finite gradient norm in {name}: {param_norm.item()}")
                    continue
                total_norm += param_norm.item() ** 2
                param_count += 1
        total_norm = total_norm ** 0.5 if param_count > 0 else 0.0
        print(f"Parameters with gradients: {param_count}, Global norm: {total_norm:.4f}")
        return total_norm

    def update_threshold(self, grad_norm):
        if not torch.isfinite(torch.tensor(grad_norm)):  # Skip infinite norms
            print(f"Warning: Skipping threshold update due to infinite grad_norm: {grad_norm}")
            return
        self.grad_norm_history.append(grad_norm)
        ma_grad_norm = sum(self.grad_norm_history) / len(self.grad_norm_history)
        clipped_norm = min(max(ma_grad_norm, 0.1), 100.0)  # Tighter upper bound
        self.current_threshold = max(1.0, min(50.0, clipped_norm * self.alpha + (1 - self.alpha) * self.current_threshold))
        print(f"Updated grad_norm_threshold: {self.current_threshold:.4f} (MA grad_norm: {ma_grad_norm:.4f})")

    def step(self, epoch, global_norm):
        self.current_epoch = epoch
        self.step_count += 1

        # Warmup: Keep all layers at FP32
        if self.step_count <= self.warmup_steps:
            for idx, layer in enumerate(self.adaptable_layers):
                if layer.current_precision != 32:
                    layer.set_precision(32)
                    self._log_switch(idx, 32, global_norm, "increase")
                    print(f"⚡ Warmup | Layer {idx} set to 32-bit (grad_norm={global_norm:.4f})")
            return global_norm

        # Update threshold and cooldowns
        self.update_threshold(global_norm)
        for i in range(len(self.layer_cooldowns)):
            if self.layer_cooldowns[i] > 0:
                self.layer_cooldowns[i] -= 1

        # Adjust precision for all layers
        ma_grad_norm = sum(self.grad_norm_history) / len(self.grad_norm_history) if self.grad_norm_history else global_norm
        for idx, layer in enumerate(self.adaptable_layers):
            if self.layer_cooldowns[idx] > 0:
                continue
            current_precision = layer.current_precision
            new_precision = current_precision

            # Increase precision if gradients are high
            if torch.isfinite(torch.tensor(ma_grad_norm)) and ma_grad_norm > self.current_threshold * 2.0:  # Stricter increase
                if current_precision == 4:
                    new_precision = 8
                elif current_precision == 8:
                    new_precision = 32
            # Decrease precision if gradients are stable
            elif torch.isfinite(torch.tensor(ma_grad_norm)) and ma_grad_norm < self.current_threshold * 0.3:  # Stricter decrease
                if current_precision == 32:
                    new_precision = 8
                elif current_precision == 8:
                    new_precision = 4
            # Periodic attempt to decrease precision to INT8 only
            elif self.step_count % 500 == 0 and current_precision > 8:
                new_precision = 8

            if new_precision != current_precision:
                layer.set_precision(new_precision)
                self.layer_cooldowns[idx] = self.cooldown_steps
                self._log_switch(idx, new_precision, ma_grad_norm, "increase" if new_precision > current_precision else "decrease")
                print(f"⚡ Epoch {self.current_epoch} | Layer {idx} switched to {new_precision}-bit (MA grad_norm={ma_grad_norm:.4f})")

        return global_norm

    def _log_switch(self, layer_idx, new_prec, norm, switch_type):
        event = {"layer_id": layer_idx, "new_precision": new_prec, "grad_norm": norm, "epoch": self.current_epoch, "type": switch_type}
        self.switch_log.append(event)

def build_adaptive_quantized_model(device, model_type, task_type):
    print(f"\n🏗️ Building Adaptive Quantized {model_type.capitalize()} Model...")
    ModelClass = BertForSequenceClassification if model_type == 'bert' else DistilBertForSequenceClassification
    ModelBase = "bert-base-uncased" if model_type == 'bert' else "distilbert-base-uncased"
    model = ModelClass.from_pretrained(ModelBase, num_labels=2).to(device)

    class QuantizedModel(nn.Module):
        def __init__(self, model_to_wrap, model_type_local):
            super().__init__()
            self.model = model_to_wrap
            self.config = self.model.config
            layers_to_quantize = self.model.bert.encoder.layer if model_type_local == "bert" else self.model.distilbert.transformer.layer
            for i in range(len(layers_to_quantize)):
                if i in [0, len(layers_to_quantize)-1]:
                    continue
                layers_to_quantize[i] = AdaptiveQuantizedLayer(layers_to_quantize[i], quant_bits=32)
        def forward(self, **kwargs):
            return self.model(**kwargs)

    quantized_model = QuantizedModel(model, model_type).to(device)
    print(f"✓ Successfully created quantized {model_type.capitalize()} model.")
    return quantized_model
# --- SECTION: 3. DATA LOADING ---
def get_dataloaders(model_type, seq_len, batch_size):
    from transformers import DistilBertTokenizerFast, BertTokenizerFast
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') if model_type == 'distilbert' else BertTokenizerFast.from_pretrained('bert-base-uncased')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def preprocess_glue_sst2(examples):
        enc = tokenizer(examples["sentence"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    def preprocess_glue_mrpc(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    def preprocess_glue_rte(examples):
        enc = tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", max_length=seq_len, truncation=True)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": examples["label"]}

    try:
        glue_sst2_train_data = load_dataset("glue", "sst2", split=f"train[:{MAX_SAMPLES}]")
        glue_sst2_validation_data = load_dataset("glue", "sst2", split=f"validation[:{MAX_SAMPLES}]")
        glue_mrpc_train_data = load_dataset("glue", "mrpc", split=f"train[:{5000}]")
        glue_mrpc_validation_data = load_dataset("glue", "mrpc", split=f"validation[:{5000}]")
        glue_rte_train_data = load_dataset("glue", "rte", split=f"train[:{2490}]")
        glue_rte_validation_data = load_dataset("glue", "rte", split=f"validation[:{277}]")

        glue_sst2_train_data = glue_sst2_train_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_sst2_validation_data = glue_sst2_validation_data.map(preprocess_glue_sst2, batched=True, remove_columns=['sentence', 'idx', 'label'])
        glue_mrpc_train_data = glue_mrpc_train_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_mrpc_validation_data = glue_mrpc_validation_data.map(preprocess_glue_mrpc, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_train_data = glue_rte_train_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])
        glue_rte_validation_data = glue_rte_validation_data.map(preprocess_glue_rte, batched=True, remove_columns=['sentence1', 'sentence2', 'idx', 'label'])

        glue_sst2_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_sst2_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_mrpc_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_mrpc_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_rte_train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        glue_rte_validation_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

        dataloaders = {
            'glue_sst2_train': DataLoader(glue_sst2_train_data, batch_size=batch_size, shuffle=True),
            'glue_sst2_validation': DataLoader(glue_sst2_validation_data, batch_size=batch_size, shuffle=False),
            'glue_mrpc_train': DataLoader(glue_mrpc_train_data, batch_size=batch_size, shuffle=True),
            'glue_mrpc_validation': DataLoader(glue_mrpc_validation_data, batch_size=batch_size, shuffle=False),
            'glue_rte_train': DataLoader(glue_rte_train_data, batch_size=batch_size, shuffle=True),
            'glue_rte_validation': DataLoader(glue_rte_validation_data, batch_size=batch_size, shuffle=False),
        }
        return dataloaders, tokenizer
    except Exception as e:
        print(f"⚠️ get_dataloaders failed: {e}. Returning empty dataloaders.")
        return {}, None

def evaluate_classification(model, dataloader, device, is_bert):
    model.eval()
    predictions, labels = [], []
    for batch in tqdm(dataloader, desc="Evaluating", leave=False):
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        if not is_bert:
            inputs.pop('token_type_ids', None)
        if DEVICE_CONFIG['mixed_precision'] and device.type == 'cuda':
            inputs = {k: v.half() if v.dtype == torch.float32 else v for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        predicted = torch.argmax(logits, dim=-1)
        predictions.extend(predicted.cpu().numpy())
        labels.extend(batch['labels'].cpu().numpy())
    return {"accuracy": accuracy_score(labels, predictions), "f1": f1_score(labels, predictions, average='weighted')}

# --- SECTION: 5. FINE-TUNING ---
def fine_tune(model, train_dataloader, validation_dataloader, device, epochs=5, task_type="classification", is_bert=False):
    from torch.cuda.amp import autocast, GradScaler
    scaler = GradScaler() if DEVICE_CONFIG['mixed_precision'] and device.type == 'cuda' else None
    model.train()
    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Added weight decay
    num_training_steps = len(train_dataloader) * epochs
    num_warmup_steps = int(num_training_steps * 0.2)
    lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

    precision_scheduler = None
    if any(isinstance(m, AdaptiveQuantizedLayer) for m in model.modules()):
        precision_scheduler = PrecisionScheduler(model, initial_grad_norm_threshold=10.0, ma_window_size=20, cooldown_steps=20, warmup_steps=1000 if task_type == 'sst2' else 500, task_name=task_type)

    for layer in model.modules():
        if isinstance(layer, AdaptiveQuantizedLayer):
            layer.set_precision(32)

    best_f1 = 0.0
    patience = 3
    patience_counter = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", leave=False):
            optimizer.zero_grad()
            inputs = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            if 'input_ids' not in inputs:
                print("Skipping a batch with missing 'input_ids'.")
                continue
            with autocast() if scaler else torch.no_grad():
                outputs = model(**inputs)
            loss = outputs.loss
            total_loss += loss.item()
            if scaler:
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                unclipped_grad_norm = precision_scheduler.get_global_norm() if precision_scheduler else 0.0
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
                if precision_scheduler:
                    precision_scheduler.step(epoch + 1, unclipped_grad_norm)
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                unclipped_grad_norm = precision_scheduler.get_global_norm() if precision_scheduler else 0.0
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
                if precision_scheduler:
                    precision_scheduler.step(epoch + 1, unclipped_grad_norm)
                optimizer.step()
            lr_scheduler.step()

        print(f"Average loss: {total_loss / len(train_dataloader):.4f}")

        # Validation after each epoch
        metrics = evaluate_classification(model, validation_dataloader, device, is_bert)
        print(f"Validation - Accuracy: {metrics['accuracy']:.4f}, F1: {metrics['f1']:.4f}")

        # Early stopping
        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    return precision_scheduler.switch_log if precision_scheduler else []

# --- SECTION 6: SINGLE TASK EXPERIMENT RUNNER ---
def run_single_task_experiment(model_config, task_name, device):
    model_type = model_config['model_type']
    is_quantized = model_config['is_quantized']
    model_name_prefix = model_config['name']

    model_full_name = f"{model_name_prefix}_{task_name.upper()}"
    print(f"\n{'='*20} Running Experiment: {model_full_name} {'='*20}")

    dataloaders, tokenizer = get_dataloaders(model_type, SEQ_LENGTH_CLASSIFICATION, BATCH_SIZE)
    train_dataloader = dataloaders.get(f'glue_{task_name}_train')
    validation_dataloader = dataloaders.get(f'glue_{task_name}_validation')
    if not train_dataloader or not validation_dataloader:
        print(f"⚠️ Dataloaders for task '{task_name}' not found. Skipping.")
        return None

    if is_quantized:
        model = build_adaptive_quantized_model(device, model_type, task_name)
    else:
        ModelClass = DistilBertForSequenceClassification if model_type == 'distilbert' else BertForSequenceClassification
        model = ModelClass.from_pretrained(f"{model_type}-base-uncased", num_labels=2).to(device)

    start_time = time.time()
    tracker = OfflineEmissionsTracker(project_name=model_full_name, country_iso_code="USA")
    tracker.start()

    epochs = 5 if task_name == 'rte' else 5
    adaptation_log = fine_tune(model, train_dataloader, validation_dataloader, device, epochs=epochs, task_type=task_name, is_bert=(model_type=='bert'))

    metrics = evaluate_classification(model, validation_dataloader, device, is_bert=(model_type=='bert'))

    emissions_kwh = tracker.stop() or 0.0
    total_duration_s = time.time() - start_time

    results = {'model_name': model_full_name, 'dataset': task_name.upper()}
    results.update(metrics)

    def get_scheduler_summary(model, log):
        summary = {
            'adaptation_log': log,
            'precision_switch_counts_int4': 0,
            'precision_switch_counts_int8': 0,
            'precision_switch_counts_fp32': 0,
            'avg_precision_level': 32.0,
            'precision_distribution': {'INT4': 0.0, 'INT8': 0.0, 'FP32': 1.0}
        }
        quant_layers = [m for m in model.modules() if isinstance(m, AdaptiveQuantizedLayer)]
        if not quant_layers:
            return summary
        total_layers = len(quant_layers)
        int4_count = sum(1 for l in quant_layers if l.current_precision == 4)
        int8_count = sum(1 for l in quant_layers if l.current_precision == 8)
        fp32_count = sum(1 for l in quant_layers if l.current_precision == 32)
        summary['avg_precision_level'] = sum(l.current_precision for l in quant_layers) / total_layers
        summary['precision_distribution'] = {
            'INT4': int4_count / total_layers,
            'INT8': int8_count / total_layers,
            'FP32': fp32_count / total_layers
        }
        summary['precision_switch_counts_int4'] = sum(l.switch_count['INT4'] for l in quant_layers)
        summary['precision_switch_counts_int8'] = sum(l.switch_count['INT8'] for l in quant_layers)
        summary['precision_switch_counts_fp32'] = sum(l.switch_count['FP32'] for l in quant_layers)
        return summary

    if is_quantized:
        results['scheduler_metrics'] = get_scheduler_summary(model, adaptation_log)

    num_queries = len(validation_dataloader.dataset)
    total_tokens_processed = num_queries * SEQ_LENGTH_CLASSIFICATION
    total_carbon_g = emissions_kwh * CARBON_INTENSITY
    results['performance_metrics'] = {
        'total_duration_s': total_duration_s,
        'total_emissions_kwh': emissions_kwh,
        'latency_ms_query': (total_duration_s / num_queries) * 1000 if num_queries > 0 else 0,
        'throughput_tokens_sec': total_tokens_processed / total_duration_s if total_duration_s > 0 else 0,
        'energy_wh_token': (emissions_kwh * 1000) / total_tokens_processed if total_tokens_processed > 0 else 0,
        'sci_gco2e_query': total_carbon_g / num_queries if num_queries > 0 else 0,
        'wue_avg_liters_query': (emissions_kwh * WATER_USAGE_FACTORS['average_l_per_kwh']) / num_queries if num_queries > 0 else 0
    }

    print("\n--- Final Results ---")
    print(json.dumps(results, indent=2))
    return results

# --- SECTION: 7. MAIN EXECUTION ---
if __name__ == "__main__":
    DEVICE = get_device()
    set_seed(42)

    all_results = []

    model_configs = [
       # {'model_type': 'distilbert', 'task_type': 'classification', 'is_quantized': True, 'name': 'Adaptive_Quantized_DistilBERT'},
       # {'model_type': 'distilbert', 'task_type': 'classification', 'is_quantized': False, 'name': 'Baseline_DistilBERT'},
     {'model_type': 'bert', 'task_type': 'classification', 'is_quantized': True, 'name': 'Adaptive_Quantized_BERT'},
        # {'model_type': 'bert', 'task_type': 'classification', 'is_quantized': False, 'name': 'Baseline_BERT'},
    ]

    tasks_to_run = ['sst2', 'mrpc', 'rte']

    for config in model_configs:
        for task in tasks_to_run:
            result = run_single_task_experiment(model_config=config, task_name=task, device=DEVICE)
            if result:
                all_results.append(result)

    if all_results:
        results_df = pd.json_normalize(all_results, sep='_')
        all_cols = {col for res in all_results for col in pd.json_normalize(res, sep='_').columns}
        results_df = results_df.reindex(columns=list(all_cols))
        results_df.to_csv("all_task_results.csv", index=False)
        with open('adaptive_quantization_results.json', 'w') as f:
            json.dump(all_results, f, indent=2)
        print("\n✅ All experiments complete. Results saved to 'all_task_results.csv'")

2025-09-25 08:06:43.537533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758787603.560816    1243 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758787603.568078    1243 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree

Using device: cuda
✓ Using GPU: Tesla T4 (CUDA)
✓ Mixed precision (FP16) enabled.

==================== Running Experiment: Adaptive_Quantized_BERT_SST2 ====================


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

Validation - Accuracy: 0.5993, F1: 0.5995


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

[codecarbon INFO @ 08:17:31] Energy consumed for RAM : 0.000740 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 08:17:31] Delta energy consumed for CPU with constant : 0.000157 kWh, power : 42.5 W
[codecarbon INFO @ 08:17:31] Energy consumed for All CPU : 0.001573 kWh
[codecarbon INFO @ 08:17:31] Energy consumed for all GPUs : 0.002816 kWh. Total GPU Power : 76.30913444911833 W
[codecarbon INFO @ 08:17:31] 0.005130 kWh of electricity used since the beginning.



--- Final Results ---
{
  "model_name": "Adaptive_Quantized_BERT_RTE",
  "dataset": "RTE",
  "accuracy": 0.5992779783393501,
  "f1": 0.5995078190306681,
  "scheduler_metrics": {
    "adaptation_log": [],
    "precision_switch_counts_int4": 0,
    "precision_switch_counts_int8": 0,
    "precision_switch_counts_fp32": 0,
    "avg_precision_level": 32.0,
    "precision_distribution": {
      "INT4": 0.0,
      "INT8": 0.0,
      "FP32": 1.0
    }
  },
  "performance_metrics": {
    "total_duration_s": 134.5682978630066,
    "total_emissions_kwh": 0.0018952341736672006,
    "latency_ms_query": 485.80612946933786,
    "throughput_tokens_sec": 263.47959038684553,
    "energy_wh_token": 5.3453129898104715e-05,
    "sci_gco2e_query": 0.0017105001567393509,
    "wue_avg_liters_query": 1.2315601128523325e-05
  }
}

✅ All experiments complete. Results saved to 'all_task_results.csv'


In [ ]:
# Save all results to a JSON file
with open('adaptive_quantization_results.json', 'w') as f:
    json.dump(all_results, f, indent=2)